# Test llcreader against reading prepared zarr output

The punchline: llcreader is comparable to zarr, so I'll roll with this because it requires less processing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from dask.distributed import Client

import pych.pigmachine as pm

from new_smooth_store import open_smoothdataset

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:38893 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 28 Memory: 67.44 GB


## llcreader for reading LLC smooth output

In [3]:
ds = open_smoothdataset('/scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/',
                        geometry='llc',
                        k_chunksize=50
                       )

In [4]:
ds.smooth3Dfld001

,Array,Chunk
Bytes,21.06 GB,4.86 MB
Shape,"(1000, 50, 13, 90, 90)","(1, 50, 3, 90, 90)"
Count,21000 Tasks,5000 Chunks
Type,float32,numpy.ndarray


### Tests with different `k_chunksize`s 

k chunksize=10 and face = 13

This never works out... just keep face chunks as is

In [7]:
ds['smooth3Dfld001'] = ds.smooth3Dfld001.chunk({'face':13})

In [8]:
%%time
for i in range(3):
    sample_mean = ds.smooth3Dfld001.mean('time').compute()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:32860 remote=tcp://127.0.0.1:33658>
distributed.nanny - WARNING - Worker exceeded 97% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 97% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.core - ERROR - "('astype-reshape-smooth3Dfld001-concatenate-b735aa77836501607e12a1c32156e3c0', 293, 4, 1, 0, 0)"
Traceback (most recent call last):
  File "/scratch2/tsmith/miniconda3/envs/py38_tim/lib/python3.8/site-packages/distributed/core.py", line 597, in handle_stream
    handler(**merge(extra, msg))
  File "/scratch2/tsmith/miniconda3/envs/py38_tim/lib/python3.8/site-packages/distributed/scheduler.py", line 2731, in handle_long_running
    ts = self.tasks[key]
KeyError: "('astype-reshape-smooth3Dfld001-concatenate-b735aa77836501607e12a1c32156e3c0', 293, 4, 1, 0, 0)"


OSError: Timed out trying to connect to 'tcp://127.0.0.1:33658' after 10 s: Timed out trying to connect to 'tcp://127.0.0.1:33658' after 10 s: connect() didn't finish in time

distributed.core - ERROR - "('astype-reshape-smooth3Dfld001-concatenate-b735aa77836501607e12a1c32156e3c0', 164, 0, 3, 0, 0)"
Traceback (most recent call last):
  File "/scratch2/tsmith/miniconda3/envs/py38_tim/lib/python3.8/site-packages/distributed/core.py", line 597, in handle_stream
    handler(**merge(extra, msg))
  File "/scratch2/tsmith/miniconda3/envs/py38_tim/lib/python3.8/site-packages/distributed/scheduler.py", line 2731, in handle_long_running
    ts = self.tasks[key]
KeyError: "('astype-reshape-smooth3Dfld001-concatenate-b735aa77836501607e12a1c32156e3c0', 164, 0, 3, 0, 0)"
distributed.utils - ERROR - "('astype-reshape-smooth3Dfld001-concatenate-b735aa77836501607e12a1c32156e3c0', 164, 0, 3, 0, 0)"
Traceback (most recent call last):
  File "/scratch2/tsmith/miniconda3/envs/py38_tim/lib/python3.8/site-packages/distributed/utils.py", line 656, in log_errors
    yield
  File "/scratch2/tsmith/miniconda3/envs/py38_tim/lib/python3.8/site-packages/distributed/scheduler.py", line 18

k chunksize=50

Sometimes, I've seen weird dask stuff happen... but this looks pretty good to me...

In [7]:
%%time
for i in range(3):
    sample_mean = ds.smooth3Dfld001.mean('time').compute()

CPU times: user 1min 21s, sys: 4.52 s, total: 1min 25s
Wall time: 1min 44s


In [7]:
%%time
for i in range(3):
    sample_mean = ds.smooth3Dfld001.mean('time').compute()

CPU times: user 1min 8s, sys: 4.8 s, total: 1min 13s
Wall time: 1min 38s


k chunksize=25

In [7]:
%%time
for i in range(3):
    sample_mean = ds.smooth3Dfld001.mean('time').compute()

distributed.scheduler - ERROR - Couldn't gather keys {"('concatenate-5a8afcf6ac8933c82a95ea6059cf5350', 1)": [], "('concatenate-5a8afcf6ac8933c82a95ea6059cf5350', 0)": []} state: ['waiting', 'waiting'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('concatenate-5a8afcf6ac8933c82a95ea6059cf5350', 1)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('concatenate-5a8afcf6ac8933c82a95ea6059cf5350', 0)
NoneType: None


CPU times: user 2min 8s, sys: 7.09 s, total: 2min 15s
Wall time: 3min 8s


k_chunksize=10

In [7]:
%%time
for i in range(3):
    sample_mean = ds.smooth3Dfld001.mean('time').compute()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:58074 remote=tcp://127.0.0.1:38431>


CPU times: user 3min 50s, sys: 9.39 s, total: 3min 59s
Wall time: 4min


Now zstore with all 50 k levels as single chunk

In [3]:
ds2 = xr.open_zarr('/scratch2/tsmith/generic-matern-covariance/sampling/llc90/sample-3D-C/zstores/matern.10dx.01xi.zarr',
                   chunks={'k':None})

In [4]:
%%time
for i in range(3):
    sample_mean = ds2.smooth3Dfld001.mean('sample').compute()

CPU times: user 29.6 s, sys: 2.53 s, total: 32.1 s
Wall time: 59.9 s


each k level as a single chunk

In [3]:
ds3 = xr.open_zarr('/scratch2/tsmith/generic-matern-covariance/sampling/llc90/sample-3D-C/zstores/matern.10dx.01xi.zarr',
                  )

In [4]:
%%time
for i in range(3):
    sample_mean = ds3.smooth3Dfld001.mean('sample').compute()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:59664 remote=tcp://127.0.0.1:40596>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:59666 remote=tcp://127.0.0.1:40596>


CPU times: user 5min 23s, sys: 12.1 s, total: 5min 35s
Wall time: 6min 27s


Middle... 10 k levels per chunk

In [3]:
ds4 = xr.open_zarr('/scratch2/tsmith/generic-matern-covariance/sampling/llc90/sample-3D-C/zstores/matern.10dx.01xi.zarr',
                   chunks={'k':10}
                  )

/scratch2/tsmith/miniconda3/envs/py38_tim/lib/python3.8/site-packages/xarray/backends/zarr.py:716: UserWarning: Specified Dask chunks 10 would separate Zarr chunk shape 50 for dimension 'k'. This significantly degrades performance. Consider rechunking after loading instead.
  chunk_spec = get_chunk(name, var, chunks)


In [4]:
%%time
for i in range(3):
    sample_mean = ds4.smooth3Dfld001.mean('sample').compute()

CPU times: user 47.4 s, sys: 3.14 s, total: 50.6 s
Wall time: 1min 1s


In [29]:
21000/5000

4.2

In [26]:
1050000 / 250000

4.2

## Compare to `mds_store`

Just looking at number of chunks here

In [8]:
from new_smooth_store import open_smoothdataset

In [9]:
ds = open_smoothdataset(run_dir, prefix=['smooth3Dfld001'], geometry='llc')

/home/tsmith/xmitgcm/xmitgcm/mds_store.py:922: UserWarning: Couldn't find available_diagnostics.log in /scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/ or /scratch2/tsmith/generic-matern-covariance/sampling/llc90/testwrite-3D-C/run.10dx.01xi/. Using default version.
  warnings.warn("Couldn't find available_diagnostics.log "


In [11]:
ds.smooth3Dfld001

<xarray.DataArray 'smooth3Dfld001' (sample: 1000, k: 50, face: 13, j: 90, i: 90)>
dask.array<concatenate, shape=(1000, 50, 13, 90, 90), dtype=float32, chunksize=(1, 1, 1, 90, 90), chunktype=numpy.ndarray>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    YC       (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    CS       (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    SN       (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    Z        (k) >f4 dask.array<chunksize=(50,), meta=np.ndarray>
    rA       (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    Depth    (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    drF      (k) >f4 dask.array<chunksize=(50,), meta=np.ndarray>
    PHrefC   (k) >f4 dask.array<chunksize=(50,), meta=np.ndarray>
    hFacC    (k, face, j, i) >f4 dask.array<chunksize=(1, 1, 90, 90), meta=np.ndarray>
    maskC    (k, face, j, i) bool dask.array<chunksize=(1, 1, 90, 90), meta=np.ndarray>
    rhoRef   (k) >f4 dask.array<chunksize=(50,), meta=np.ndarray>
    dxF      (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    dyF      (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    rLowC    (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
    rSurfC   (face, j, i) >f4 dask.array<chunksize=(1, 90, 90), meta=np.ndarray>
  * sample   (sample) int64 0 1 2 3 4 5 6 7 ... 992 993 994 995 996 997 998 999
Attributes:
    standard_name:  smooth_fld
    long_name:      $C\mathbf{z}$

In [30]:
3250000 / 650000

5.0